# December 18th

We're doing the area inside an arbitrary path again. Input is 784 lines which will translate to 784 path segments.

In [1]:
example = [
    "R 6 (#70c710)",
    "D 5 (#0dc571)",
    "L 2 (#5713f0)",
    "D 2 (#d2c081)",
    "R 2 (#59c680)",
    "D 2 (#411b91)",
    "L 5 (#8ceee2)",
    "U 2 (#caa173)",
    "L 1 (#1b58a2)",
    "U 2 (#caa171)",
    "R 2 (#7807d2)",
    "U 3 (#a77fa3)",
    "L 2 (#015232)",
    "U 2 (#7a21e3)"
]

examplepartoneresult = 62


62

Even though they're showing us this on a grid we can just treat these as arbitrary path segments. Let's turn our input into a collection of path segements.

In [2]:
struct PathSegment
    vector::Tuple{AbstractRange{Int},AbstractRange{Int}}
end

function parsecontents(contents)
    segements = []
    start = (0, 0)
    for line in contents
        (d, s) = split(line)
        dv = if d == "U"
            (-1, 0)
        elseif d == "D"
            (1, 0)
        elseif d == "R"
            (0, 1)
        else
            (0, -1)
        end
        stop = start .+ (dv .* parse(Int, s))
        push!(segements, PathSegment((
            range(start[1], stop[1], step=dv[1] == 0 ? 1 : dv[1]),
            range(start[2], stop[2], step=dv[2] == 0 ? 1 : dv[2])
        )))
        start = stop
    end
    return segements
end

println(parsecontents(example))

function determinebounds(segments)
    ymin = min(segments[begin].vector[1].start, segments[begin].vector[1].stop)
    ymax = max(segments[begin].vector[1].start, segments[begin].vector[1].stop)
    xmin = min(segments[begin].vector[2].start, segments[begin].vector[2].stop)
    xmax = max(segments[begin].vector[2].start, segments[begin].vector[2].stop)
    for segment in segments
        ymin = min(segment.vector[1].start, segment.vector[1].stop, ymin)
        ymax = max(segment.vector[1].start, segment.vector[1].stop, ymax)
        xmin = min(segment.vector[2].start, segment.vector[2].stop, xmin)
        xmax = max(segment.vector[2].start, segment.vector[2].stop, xmax)
    end
    return (ymin:ymax, xmin:xmax)
end

println(determinebounds(parsecontents(example)))

Any[PathSegment((0:1:0, 0:1:6)), PathSegment((0:1:5, 6:1:6)), PathSegment((5:1:5, 6:-1:4)), PathSegment((5:1:7, 4:1:4)), PathSegment((7:1:7, 4:1:6)), PathSegment((7:1:9, 6:1:6)), PathSegment((9:1:9, 6:-1:1)), PathSegment((9:-1:7, 1:1:1)), PathSegment((7:1:7, 1:-1:0)), PathSegment((7:-1:5, 0:1:0)), PathSegment((5:1:5, 0:1:2)), PathSegment((5:-1:2, 2:1:2)), PathSegment((2:1:2, 2:-1:0)), PathSegment((2:-1:0, 0:1:0))]
(0:9, 0:6)


Now with our path segements it's time to dust off our "inside the path" function from December 10th

In [3]:
function isvertical(segement::PathSegment)
    return segement.vector[2].start == segement.vector[2].stop
end

function isonline(segments, position)
    any(
        segment -> !any(isempty, intersect.(segment.vector, position)),
        segments
    )
end

## adaptation of the even/odd rule for shape filing in vector graphics
function isenclosed(segments, position, bounds::Tuple{UnitRange{Int},UnitRange{Int}})
    ## track the number of times we cross a path segement
    intersections = 0
    ## track how many corners we could go over
    unders = 0
    ## track how many corners we could go under
    overs = 0
    ## go from the position to the left edge in a stright line
    xrange = range(position[2] - 1, bounds[2].start, step=-1)
    intersecting = filter(
        segment -> isvertical(segment) && !any(isempty, intersect.(segment.vector, (position[1], xrange))),
        segments
    )
    for segment in intersecting
        if min(segment.vector[1].start, segment.vector[1].stop) == position[1]
            unders += 1
        elseif max(segment.vector[1].start, segment.vector[1].stop) == position[1]
            overs += 1
        else
            intersections += 1
        end
    end
    ## we must cross | pipes, we have to cross a pair of corers that we can go over and under, use min to count pairs
    ## eg. |F7J <- this is 2 crossings since we go through the |, over the F and 7, and through the J
    ## if we cross an odd number of lines then we must be encosed by the shape, otherwise we are not.
    return isodd(intersections + min(unders, overs))
end

function getfilled(segments, bounds)
    yoffset = 1 - bounds[1].start
    xoffset = 1 - bounds[2].start
    grid = fill(false, (length(bounds[1]), length(bounds[2])))
    for y in bounds[1]
        for x in bounds[2]
            if isonline(segments, (y, x)) #|| isenclosed(segments, (y, x), bounds)
                grid[y+yoffset, x+xoffset] = true
            end
        end
    end
    return grid
end



getfilled (generic function with 1 method)

Part one took 20 minutes using the `isenclosed` function this seems to be driven by the large number of squares to consdier. Instead of doing it
with enclosure lets find squares to fit inside the space.

First we'll want to segment horizontal and vertical lines and sort them by their starting points

In [4]:
function vandh(segments)
    vsegments = []
    hsegments = []
    for segment in segments
        if isvertical(segment)
            push!(vsegments, segment)
        else
            push!(hsegments, segment)
        end
    end
    sort!(vsegments, lt=(s1, s2) -> s1.vector[2].start < s2.vector[2].start)
    sort!(hsegments, lt=(s1, s2) -> s1.vector[1].start < s2.vector[1].start)
    return (vsegments, hsegments)
end

vandh(parsecontents(example))

(Any[PathSegment((7:-1:5, 0:1:0)), PathSegment((2:-1:0, 0:1:0)), PathSegment((9:-1:7, 1:1:1)), PathSegment((5:-1:2, 2:1:2)), PathSegment((5:1:7, 4:1:4)), PathSegment((0:1:5, 6:1:6)), PathSegment((7:1:9, 6:1:6))], Any[PathSegment((0:1:0, 0:1:6)), PathSegment((2:1:2, 2:-1:0)), PathSegment((5:1:5, 6:-1:4)), PathSegment((5:1:5, 0:1:2)), PathSegment((7:1:7, 4:1:6)), PathSegment((7:1:7, 1:-1:0)), PathSegment((9:1:9, 6:-1:1))])

Next we'll write a recursive divide and conquer algorithm to find area inside the lines

In [17]:
function recover(coverage, normalize)
    result = []
    for i in firstindex(coverage):lastindex(coverage)
        if normalize.start == coverage[i].start || normalize.stop == coverage[i].stop
            ## strink from the bottom or top
            r = if normalize.start == coverage[i].start
                normalize.stop:coverage[i].stop
            else
                coverage[i].start:normalize.start
            end
            if length(r) > 1
                push!(result, r)
            end
            push!(result, coverage[i+1:end]...)
            return result
        end

        if normalize.start == coverage[i].stop || normalize.stop == coverage[i].start
            r = (min(coverage[i].start, normalize.start):max(coverage[i].stop, normalize.stop))
            if i < lastindex(coverage) && r.stop == coverage[i+1].start
                r = r.start:coverage[i+1].stop
                i += 1
            end
            push!(result, r, coverage[i+1:end]...)
            return result
        end

        if normalize.start > coverage[i].start && normalize.stop < coverage[i].stop
            r1 = coverage[i].start:normalize.start
            r2 = normalize.stop:coverage[i].stop
            push!(result, r1, r2, coverage[i+1:end]...)
            return result
        end

        if normalize.stop < coverage[i].start
            push!(result, normalize, coverage[i:end]...)
            return result
        end

        push!(result, coverage[i])

    end

    push!(result, normalize)
    return result

end

function takeallforx!(vsegments)
    if isempty(vsegments)
        return []
    end
    r = [popfirst!(vsegments)]
    x = r[begin].vector[2].start
    while !isempty(vsegments)
        if vsegments[begin].vector[2].start == x
            push!(r, popfirst!(vsegments))
        else
            break
        end
    end
    return r
end

function normalizerange(v)
    min(v.vector[1].start, v.vector[1].stop):max(v.vector[1].start, v.vector[1].stop)
end

function getarea(segments, bounds)
    (vsegments, _) = vandh(segments)
    coverage = []
    area = 0
    previousx = bounds[2].start
    while !isempty(vsegments)
        increment = sum(map(length, coverage), init=0)
        n = takeallforx!(vsegments)
        x = n[begin].vector[2].start
        for norm in map(normalizerange, n)
            coverage = recover(coverage, norm)
        end
        nextincrement = sum(map(length, coverage), init=0)

        factor = x - previousx + 1
        ni = min(nextincrement, increment)

        area += (increment * factor) - ni
        previousx = x
    end

    return area
end


getarea (generic function with 1 method)

## Partone
but it's super inefficient. We'll have to look into why as we get into Part2

In [6]:
function partone(contents)
    segments = parsecontents(contents)
    getarea(segments, determinebounds(segments))
end

partone(example)

62

## Part Two

So we didn't need the colors after all

In [15]:
function parsecontentparttwo(contents)
    segements = []
    start = (0, 0)
    for line in contents
        (_, _, c) = split(line)
        d = c[end-1]
        s = c[begin+2:end-2]
        dv = if d == '3'
            (-1, 0)
        elseif d == '1'
            (1, 0)
        elseif d == '0'
            (0, 1)
        else
            (0, -1)
        end
        stop = start .+ (dv .* parse(Int, s, base=16))
        push!(segements, PathSegment((
            range(start[1], stop[1], step=dv[1] == 0 ? 1 : dv[1]),
            range(start[2], stop[2], step=dv[2] == 0 ? 1 : dv[2])
        )))
        start = stop
    end
    return segements
end

parsecontentparttwo (generic function with 1 method)

In [19]:
function parttwo(contents)
    segments = parsecontentparttwo(contents)
    getarea(segments, determinebounds(segments))
end

println(parttwo(example))
println(952408144115)

952408144115
952408144115


## Result

In [20]:
include("./aoc.jl")

execute(18, partone, parttwo)

68016
71262565063800


LUCA